# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-05-05 04:40:46] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=32383, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=562545640, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_

[2025-05-05 04:40:55] Attention backend not set. Use fa3 backend by default.
[2025-05-05 04:40:55] Init torch distributed begin.


[2025-05-05 04:40:55] Init torch distributed ends. mem usage=0.00 GB
[2025-05-05 04:40:55] Load weight begin. avail mem=53.74 GB


[2025-05-05 04:40:56] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-05 04:40:56] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-05-05 04:40:59] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-05 04:41:00] INFO:     Started server process [3974579]
[2025-05-05 04:41:00] INFO:     Waiting for application startup.
[2025-05-05 04:41:00] INFO:     Application startup complete.
[2025-05-05 04:41:00] INFO:     Uvicorn running on http://0.0.0.0:32383 (Press CTRL+C to quit)


[2025-05-05 04:41:00] INFO:     127.0.0.1:44140 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-05 04:41:01] INFO:     127.0.0.1:44152 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-05 04:41:01] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-05 04:41:05] INFO:     127.0.0.1:44154 - "POST /generate HTTP/1.1" 200 OK
[2025-05-05 04:41:05] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-05 04:41:06] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-05 04:41:07] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 4.95, #queue-req: 0


[2025-05-05 04:41:08] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 97.06, #queue-req: 0


[2025-05-05 04:41:08] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 100.12, #queue-req: 0


[2025-05-05 04:41:08] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 100.55, #queue-req: 0


...

[2025-05-05 04:41:26] Decode batch. #running-req: 1, #token: 1932, token usage: 0.09, gen throughput (token/s): 98.69, #queue-req: 0


[2025-05-05 04:41:26] Decode batch. #running-req: 1, #token: 1972, token usage: 0.10, gen throughput (token/s): 95.40, #queue-req: 0


[2025-05-05 04:41:26] Decode batch. #running-req: 1, #token: 2012, token usage: 0.10, gen throughput (token/s): 105.24, #queue-req: 0


[2025-05-05 04:41:27] Decode batch. #running-req: 1, #token: 2052, token usage: 0.10, gen throughput (token/s): 74.31, #queue-req: 0


[2025-05-05 04:41:27] INFO:     127.0.0.1:40304 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-05 04:41:27] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-05 04:41:28] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, gen throughput (token/s): 52.88, #queue-req: 0


[2025-05-05 04:41:28] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, gen throughput (token/s): 74.54, #queue-req: 0


[2025-05-05 04:41:29] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, gen throughput (token/s): 72.54, #queue-req: 0


[2025-05-05 04:41:29] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, gen throughput (token/s): 65.25, #queue-req: 0


...

[2025-05-05 04:41:49] Decode batch. #running-req: 1, #token: 1924, token usage: 0.09, gen throughput (token/s): 103.92, #queue-req: 0


[2025-05-05 04:41:50] Decode batch. #running-req: 1, #token: 1964, token usage: 0.10, gen throughput (token/s): 104.21, #queue-req: 0


[2025-05-05 04:41:50] Decode batch. #running-req: 1, #token: 2004, token usage: 0.10, gen throughput (token/s): 104.35, #queue-req: 0


[2025-05-05 04:41:51] Decode batch. #running-req: 1, #token: 2044, token usage: 0.10, gen throughput (token/s): 104.46, #queue-req: 0


[2025-05-05 04:41:51] INFO:     127.0.0.1:40304 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-05 04:41:51] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-05 04:41:51] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, gen throughput (token/s): 90.08, #queue-req: 0


[2025-05-05 04:41:51] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, gen throughput (token/s): 107.01, #queue-req: 0


[2025-05-05 04:41:52] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, gen throughput (token/s): 107.73, #queue-req: 0
[2025-05-05 04:41:52] INFO:     127.0.0.1:40304 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-05 04:41:52] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-05 04:41:52] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 94.84, #queue-req: 0


[2025-05-05 04:41:53] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 106.59, #queue-req: 0


[2025-05-05 04:41:53] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, gen throughput (token/s): 103.02, #queue-req: 0


[2025-05-05 04:41:53] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, gen throughput (token/s): 105.12, #queue-req: 0


[2025-05-05 04:41:54] Decode batch. #running-req: 1, #token: 193, token usage: 0.01, gen throughput (token/s): 106.30, #queue-req: 0


[2025-05-05 04:41:54] Decode batch. #running-req: 1, #token: 233, token usage: 0.01, gen throughput (token/s): 106.51, #queue-req: 0


[2025-05-05 04:41:54] Decode batch. #running-req: 1, #token: 273, token usage: 0.01, gen throughput (token/s): 105.94, #queue-req: 0


[2025-05-05 04:41:55] Decode batch. #running-req: 1, #token: 313, token usage: 0.02, gen throughput (token/s): 106.45, #queue-req: 0


[2025-05-05 04:41:55] Decode batch. #running-req: 1, #token: 353, token usage: 0.02, gen throughput (token/s): 106.50, #queue-req: 0
[2025-05-05 04:41:55] INFO:     127.0.0.1:40304 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-05 04:41:56] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-05 04:41:56] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, gen throughput (token/s): 53.60, #queue-req: 0


[2025-05-05 04:41:56] Decode batch. #running-req: 1, #token: 535, token usage: 0.03, gen throughput (token/s): 101.25, #queue-req: 0


[2025-05-05 04:41:57] Decode batch. #running-req: 1, #token: 575, token usage: 0.03, gen throughput (token/s): 103.70, #queue-req: 0


[2025-05-05 04:41:57] Decode batch. #running-req: 1, #token: 615, token usage: 0.03, gen throughput (token/s): 95.74, #queue-req: 0


[2025-05-05 04:41:58] Decode batch. #running-req: 1, #token: 655, token usage: 0.03, gen throughput (token/s): 103.03, #queue-req: 0


[2025-05-05 04:41:58] Decode batch. #running-req: 1, #token: 695, token usage: 0.03, gen throughput (token/s): 102.16, #queue-req: 0


[2025-05-05 04:41:58] INFO:     127.0.0.1:40304 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-05-05 04:41:59] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-05 04:41:59] Decode batch. #running-req: 1, #token: 38, token usage: 0.00, gen throughput (token/s): 46.72, #queue-req: 0


[2025-05-05 04:41:59] Decode batch. #running-req: 1, #token: 78, token usage: 0.00, gen throughput (token/s): 107.46, #queue-req: 0


[2025-05-05 04:42:00] Decode batch. #running-req: 1, #token: 118, token usage: 0.01, gen throughput (token/s): 107.78, #queue-req: 0


[2025-05-05 04:42:00] Decode batch. #running-req: 1, #token: 158, token usage: 0.01, gen throughput (token/s): 106.76, #queue-req: 0


[2025-05-05 04:42:00] Decode batch. #running-req: 1, #token: 198, token usage: 0.01, gen throughput (token/s): 106.96, #queue-req: 0


...

[2025-05-05 04:42:17] Decode batch. #running-req: 1, #token: 1918, token usage: 0.09, gen throughput (token/s): 104.51, #queue-req: 0


[2025-05-05 04:42:18] Decode batch. #running-req: 1, #token: 1958, token usage: 0.10, gen throughput (token/s): 104.27, #queue-req: 0


[2025-05-05 04:42:18] Decode batch. #running-req: 1, #token: 1998, token usage: 0.10, gen throughput (token/s): 104.64, #queue-req: 0


[2025-05-05 04:42:19] Decode batch. #running-req: 1, #token: 2038, token usage: 0.10, gen throughput (token/s): 104.56, #queue-req: 0


[2025-05-05 04:42:19] INFO:     127.0.0.1:52280 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not entirely sure about all the details, but I\'ll try to think it through.\n\nFirst, I know that the capital of France is Paris. That\'s pretty much a given, right? But I should double-check that. Maybe I can recall any other capitals I know. London is the capital of the UK, Rome is Italy, and maybe Tokyo is Japan\'s. Yeah, Paris seems correct for France.\n\nNow, moving on to the population. I think Paris is a very large city, but I\'m not sure of the exact number. I remember it\'s over 3 million, but I\'m not certain. Maybe around 3.5 million? I should probably look that up, but since I can\'t right now, I\'ll go with 3,500,000 as an estimate.\n\nNext, the area. Paris is a big city, but I think it\'s not as large as Tokyo or London. Maybe around 10 square kilometers? I\'m not sure, but that seems 

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-05-05 04:42:19] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-05 04:42:19] Decode batch. #running-req: 1, #token: 15, token usage: 0.00, gen throughput (token/s): 98.57, #queue-req: 0


[2025-05-05 04:42:19] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 106.84, #queue-req: 0


[2025-05-05 04:42:20] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, gen throughput (token/s): 106.31, #queue-req: 0


[2025-05-05 04:42:20] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, gen throughput (token/s): 106.24, #queue-req: 0


[2025-05-05 04:42:20] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, gen throughput (token/s): 104.57, #queue-req: 0


...

[2025-05-05 04:42:25] Decode batch. #running-req: 1, #token: 615, token usage: 0.03, gen throughput (token/s): 104.50, #queue-req: 0


[2025-05-05 04:42:25] Decode batch. #running-req: 1, #token: 655, token usage: 0.03, gen throughput (token/s): 100.93, #queue-req: 0


[2025-05-05 04:42:25] Decode batch. #running-req: 1, #token: 695, token usage: 0.03, gen throughput (token/s): 104.03, #queue-req: 0


[2025-05-05 04:42:26] Decode batch. #running-req: 1, #token: 735, token usage: 0.04, gen throughput (token/s): 104.65, #queue-req: 0


[2025-05-05 04:42:26] Decode batch. #running-req: 1, #token: 775, token usage: 0.04, gen throughput (token/s): 102.80, #queue-req: 0
[2025-05-05 04:42:26] INFO:     127.0.0.1:45434 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-05-05 04:42:26] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-05 04:42:26] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-05 04:42:27] Decode batch. #running-req: 3, #token: 68, token usage: 0.00, gen throughput (token/s): 130.79, #queue-req: 0


[2025-05-05 04:42:27] Decode batch. #running-req: 3, #token: 188, token usage: 0.01, gen throughput (token/s): 302.39, #queue-req: 0


[2025-05-05 04:42:28] Decode batch. #running-req: 3, #token: 308, token usage: 0.02, gen throughput (token/s): 302.20, #queue-req: 0


[2025-05-05 04:42:28] Decode batch. #running-req: 3, #token: 428, token usage: 0.02, gen throughput (token/s): 298.45, #queue-req: 0


[2025-05-05 04:42:28] Decode batch. #running-req: 3, #token: 548, token usage: 0.03, gen throughput (token/s): 300.57, #queue-req: 0


[2025-05-05 04:42:29] INFO:     127.0.0.1:50474 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any other major city in France being referred to as the capital. So, I'm pretty confident that Paris is correct.\n</think>Paris is the capital of F

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-05-05 04:42:29] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-05 04:42:29] Decode batch. #running-req: 1, #token: 24, token usage: 0.00, gen throughput (token/s): 203.88, #queue-req: 0


[2025-05-05 04:42:29] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, gen throughput (token/s): 105.81, #queue-req: 0


[2025-05-05 04:42:30] Decode batch. #running-req: 1, #token: 104, token usage: 0.01, gen throughput (token/s): 107.22, #queue-req: 0


[2025-05-05 04:42:30] Decode batch. #running-req: 1, #token: 144, token usage: 0.01, gen throughput (token/s): 107.51, #queue-req: 0


...

[2025-05-05 04:42:47] Decode batch. #running-req: 1, #token: 1904, token usage: 0.09, gen throughput (token/s): 104.27, #queue-req: 0


[2025-05-05 04:42:47] Decode batch. #running-req: 1, #token: 1944, token usage: 0.09, gen throughput (token/s): 104.25, #queue-req: 0


[2025-05-05 04:42:48] Decode batch. #running-req: 1, #token: 1984, token usage: 0.10, gen throughput (token/s): 104.42, #queue-req: 0


[2025-05-05 04:42:48] Decode batch. #running-req: 1, #token: 2024, token usage: 0.10, gen throughput (token/s): 104.88, #queue-req: 0


[2025-05-05 04:42:48] INFO:     127.0.0.1:50486 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-05-05 04:42:48] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-05 04:42:48] Decode batch. #running-req: 1, #token: 30, token usage: 0.00, gen throughput (token/s): 99.23, #queue-req: 0


[2025-05-05 04:42:49] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, gen throughput (token/s): 105.97, #queue-req: 0


[2025-05-05 04:42:49] Decode batch. #running-req: 1, #token: 110, token usage: 0.01, gen throughput (token/s): 105.91, #queue-req: 0


[2025-05-05 04:42:49] Decode batch. #running-req: 1, #token: 150, token usage: 0.01, gen throughput (token/s): 105.35, #queue-req: 0


[2025-05-05 04:42:50] Decode batch. #running-req: 1, #token: 190, token usage: 0.01, gen throughput (token/s): 105.71, #queue-req: 0


...

[2025-05-05 04:42:52] Decode batch. #running-req: 1, #token: 470, token usage: 0.02, gen throughput (token/s): 106.15, #queue-req: 0


[2025-05-05 04:42:53] Decode batch. #running-req: 1, #token: 510, token usage: 0.02, gen throughput (token/s): 105.95, #queue-req: 0


[2025-05-05 04:42:53] Decode batch. #running-req: 1, #token: 550, token usage: 0.03, gen throughput (token/s): 103.05, #queue-req: 0


[2025-05-05 04:42:54] Decode batch. #running-req: 1, #token: 590, token usage: 0.03, gen throughput (token/s): 103.86, #queue-req: 0


[2025-05-05 04:42:54] Decode batch. #running-req: 1, #token: 630, token usage: 0.03, gen throughput (token/s): 104.29, #queue-req: 0
[2025-05-05 04:42:54] INFO:     127.0.0.1:59608 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.31s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.23s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.24s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris.

That's all.

Alright, so I need to figure out the information about the capital of France. From the user's message, it looks like they already know that the capital is Paris. They might be looking for more detailed information about Paris, perhaps for travel, a school project, or personal knowledge expansion. 

I should consider what aspects of Paris are important. Paris is not just a city; it's a cultural, political, and economic center. It's home to major landmarks like the Eiffel Tower, the Louvre Museum, and the Arc de Triomphe. It's also
Prompt: Give me the information of the capital of Germany.
Generated text: 
Alright, so I need to figure out the information about the capital of Germany. Hmm, okay. Let me think about this step by step.

First, I know that Germany is a country in Europe. I've heard the term "capital" refers to the seat of government, so the capital of Germa

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  describe its economy, culture, and location.600-700 words.
Okay, so I need to provide information about London as a major global city, focusing on its economy, culture, and location. The user wants a 600-700 word response. Let me start by breaking down each section.

First, the economy. London is known for its financial sector, so I should mention the City of London, major financial institutions, banks, insurance companies, and perhaps investment funds. I should also talk about its role in the global economy, maybe include some statistics like GDP or unemployment rates. Don't
Prompt: Please provide information about Paris as a major global city:
Generated text:  reasons why it's a major global city, its cultural significance, and the economic status of Paris. Please provide the information in bullet points.

Sure, here's the information about Paris as a major global city, its cultural significance,

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Okay, so I need to provide information about the capital of France in JSON format. Let me think about how to approach this.

First, I know that the capital of France is Paris. That's pretty straightforward. Now, I should gather more details to make the JSON comprehensive. Let me list out what I can think of.

 Paris is a major city in France, so I should mention its population. I think it's around 2.1 million people. Geographically, it's located in the northern part of the country, near the Seine River.

I should include some key landmarks. The Eiffel Tower is a must, as well as the Louvre Museum and Notre-Dame Cathedral. These are famous attractions that tourists visit.

Cuisine is an interesting aspect. Paris is known for its culinary scene, so I can mention popular dishes like baguette, croissant, and Béchamel sauce.

Cultural significance i

In [19]:
llm.shutdown()